# Atlantic Marine Mammals

This indicator was used in the Southeast Blueprint 2023.
Code written by Rua Mordecai and Amy Keister

In [1]:
import os
import arcpy

In [2]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
OutWorkspace = r"D:\Blueprint\2023\finalIndicatorEdits\ContinentalSoutheastBlueprint2023_FinalIndicators\ContinentalSoutheastBlueprint2023_FinalIndicators\SpatialData"

In [3]:
# define final output name
IndicatorFileName = "AtlanticMarineMammals.tif"

In [4]:
# define raster used for cell size, extent, and snapping
MarineRaster= r"D:\Blueprint\2023\extent\Marine_Extent_v1.tif"

In [5]:
# define inputs
mammal= r"D:\Blueprint\2023\Indicators\AtlanticMarineMammals\zonation\rankmap.tif"
nlcd = r"D:\Landcover\nlcd_2019_land_cover_l48_20210604\nlcd_2019_land_cover_l48_20210604.img"

In [6]:
# Sometimes arcpro is fussy about not overwriting things
arcpy.env.overwriteOutput = True

### Start analysis

In [7]:
# Set the output workspace
arcpy.env.workspace = OutWorkspace

### Bring in zonation results

In [8]:
# take zonation results times 100
out_raster = arcpy.sa.Times(mammal, 100); out_raster.save("mammal.tif")

In [9]:
# convert to integer
out_raster = arcpy.sa.Int("mammal.tif"); out_raster.save("indicator1.tif")

In [10]:
# Reclassify into bins and align with Marine extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=MarineRaster, snapRaster=MarineRaster, cellSize=MarineRaster):
    out_raster = arcpy.sa.Reclassify('indicator1.tif', "Value", "0 10 1;11 20 2;21 30 3;31 40 4;41 50 5;51 60 6;61 70 7;71 80 8;81 90 9;91 100 10", "DATA"); out_raster.save("binned.tif")

In [26]:
# Make an water mask in NLCD to change all land pixels to 0
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=MarineRaster, snapRaster=MarineRaster, cellSize=MarineRaster):
    out_raster = arcpy.sa.Reclassify(nlcd, "Value", "0 11 1;12 95 0"); out_raster.save("nlcdWaterMask.tif")

In [28]:
# use mask to turn land values to 0
out_raster = arcpy.sa.Times("nlcdWaterMask.tif", "binned.tif"); out_raster.save(IndicatorFileName)

### Finalize indicator



In [8]:
# set code block for next step
codeblock = """
def Reclass(v):
    if v == 10:
        return '10 = >90th percentile of importance for marine mammal index species (across the full East Coast study area)'
    elif v == 9:
        return '9 = >80th-90th percentile of importance'  
    elif v == 8:
        return '8 = >70th-80th percentile of importance'  
    elif v == 7:
        return '7 = >60th-70th percentile of importance'
    elif v == 6:
        return '6 = >50th-60th percentile of importance'
    elif v == 5:
        return '5 = >40th-50th percentile of importance'
    elif v == 4:
        return '4 = >30th-40th percentile of importance'  
    elif v == 3:
        return '3 = >20th-30th percentile of importance'  
    elif v == 2:
        return '2 = >10th-20th percentile of importance'
    elif v == 1:
        return '1 = ≤10th percentile of importance'
    elif v == 0:
        return '0 = Land'
"""

In [9]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(IndicatorFileName, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'AtlanticMarineMammals.tif'>

In [31]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 10:
		return 217
	if Value == 9:
		return 225
	if Value == 8:
		return 233
	if Value == 7:
		return 240
	if Value == 6:
		return 244
	if Value == 5:
		return 247
	if Value == 4:
		return 250
	if Value == 3:
		return 251
	if Value == 2:
		return 253
	if Value == 1:
		return 255
	else:
		return 255
		
def Reclass2(Value):
	if Value == 10:
		return 52
	if Value == 9:
		return 73
	if Value == 8:
		return 102
	if Value == 7:
		return 131
	if Value == 6:
		return 157
	if Value == 5:
		return 181
	if Value == 4:
		return 204
	if Value == 3:
		return 227
	if Value == 2:
		return 247
	if Value == 1:
		return 255
	else:
		return 255
		
def Reclass3(Value):
	if Value == 10:
		return 139
	if Value == 9:
		return 129
	if Value == 8:
		return 119
	if Value == 7:
		return 113
	if Value == 6:
		return 113
	if Value == 5:
		return 120
	if Value == 4:
		return 136
	if Value == 3:
		return 158
	if Value == 2:
		return 185
	if Value == 1:
		return 217
	else:
		return 255
		
"""

In [33]:
# calculate Red field
arcpy.management.CalculateField(IndicatorFileName, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(IndicatorFileName, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(IndicatorFileName, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'AtlanticMarineMammals.tif'>